In [1]:
using NIfTI
using Interpolations
using LinearAlgebra
using Revise
using Statistics
using Images

┌ Info: Precompiling NIfTI [a3a9e032-41b5-5fc4-967a-a6b7a19844d3]
└ @ Base loading.jl:1278
┌ Info: Precompiling Interpolations [a98d9a8b-a2ab-59e6-89dd-64a1c18fca59]
└ @ Base loading.jl:1278
┌ Info: Precompiling Images [916415d5-f1e6-5110-898d-aaa5f9f070e0]
└ @ Base loading.jl:1278


In [52]:
m = niread("../data/subject.nii.gz")
f = niread("../data/subject.nii.gz")

256×256×256 NIVolume{Float32,3,Array{Float32,3}}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0

In [6]:
struct Config
    kernel_width::Int8
    search_radius::Int8
end

In [53]:
config = Config(3, 3)
kernel_edge = (config.kernel_width-1)÷2

1

In [54]:
H(v) = v .- Statistics.mean(v)
S(w,v) = w'*H(v) / sqrt(v'*H(v))

S (generic function with 1 method)

In [55]:
padded_f = padarray(f, Pad(
        ((config.kernel_width-1)÷ 2),
        ((config.kernel_width-1)÷ 2),
        ((config.kernel_width-1)÷ 2)
))
padded_m = padarray(m, Pad(
        ((config.kernel_width-1)÷ 2 + config.search_radius),
        ((config.kernel_width-1)÷ 2 + config.search_radius),
        ((config.kernel_width-1)÷ 2 + config.search_radius)
))

264×264×264 OffsetArray(::Array{Float32,3}, -3:260, -3:260, -3:260) with eltype Float32 with indices -3:260×-3:260×-3:260:
[:, :, -3] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  

In [56]:
xdim, ydim, zdim = size(f)

(256, 256, 256)

In [ ]:
vector_field = zeros(xdim, ydim, zdim, 3)
for i in CartesianIndices(size(f))
    
    kernel_box = padded_f[i[1]-kernel_edge : i[1] + kernel_edge,
                          i[2] - kernel_edge : i[2] + kernel_edge,
                          i[3] - kernel_edge : i[3] + kernel_edge]
    
    
    max_S = nothing
    best_q = nothing
    for j = CartesianIndices((
            -config.search_radius:config.search_radius,
            -config.search_radius:config.search_radius,
            -config.search_radius:config.search_radius))
        
        # 
        q = i + j
        m_box = padded_m[q[1] - kernel_edge : q[1] + kernel_edge,
                           q[2] - kernel_edge : q[2] + kernel_edge,
                           q[3] - kernel_edge : q[3] + kernel_edge]
        
        S_val = S(vec(m_box),vec(kernel_box))
        
        if max_S == nothing
            max_S = S_val
            best_q = q
        elseif S_val > max_S
            max_S = S_val
            best_q = q
        end
        
    end
    
    vector_field[i,1] = best_q[1]
    vector_field[i,2] = best_q[2]
    vector_field[i,3] = best_q[3]
    
end